<a href="img/EOandGRASSGIS.png">
  <img src="https://github.com/veroandreo/foss4g2024_grass4rs/blob/main/img/EOandGRASSGIS.png?raw=1"
   width="25%" style="float:right">
</a>

# **<span style='color:Green'>GRASS GIS for Earth Observation data processing with Jupyter notebooks</span>**

### Workshop at FOSS4G 2024, Belem (Brazil)

*Lecturer:* Veronica Andreo

*Date:* 2024-12-02

Special acknowledgements to **Markus Neteler** and **Maris Nartiss** who were part of an earlier edition of this workshop and also prepared the Sentinel-2 dataset 🙌

---

### Foreword

This notebook will demonstrate the use of **GRASS GIS 8.4+** in combination with Python within a Jupyter Notebook in the [Google Colab](https://colab.research.google.com) environment. We will use GRASS tools and python libraries that facilitate scripting (`grass.script`) and connection/interaction with Jupyter Notebooks (`grass.jupyter`).

The workflow that will be demonstrated on this notebook ranges from searching satellite data to time series building and supervised classification.


# What is Colab?

Perhaps you have heard of Google Colaboratory or simply Colab. This is a hosted Jupyter Notebook service that requires no setup or configuration to use and provides free access to computing resources, including GPUs and TPUs.

Colab is especially well suited to machine learning, data science, and education. Furthermore, it allows easy sharing of workflows which facilitates reproducibility.

Colab notebooks allow you to combine executable code and rich text in a single document, along with images, HTML, LaTeX and more. When you create your own Colab notebooks, they are stored in your Google Drive account. You can easily share your Colab notebooks with colleagues, allowing them to comment on your notebooks or even edit them.


> See Colab's FAQ for more details: <https://research.google.com/colaboratory/faq.html> and follow the Google Colab blog on Medium at <https://medium.com/google-colab>.


## Jupyter Notebooks

Jupyter Notebooks are server-client applications that allow code written in a notebook document to be **edited and executed through a web browser**. They can be run on a local computer or used to control computations on a remote server
([see the documentation](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)).

Jupyter Notebooks are interactive and they allow to combine live code, text, and computational results in a single document. They are:

* convenient for initial code development (prototyping)
* ideal for code segmentation with the ability to re-run cells
* able to store values of variables from already executed cells

### Editing and interactive use

Editing a Jupyter Notebook is very easy: in the web browser, you can navigate between text or code cells using the mouse or keyboard shortcuts (see Tools > Keyboard Shortcuts in Colab). You can execute small code chunks cell by cell, save the notebook in its current state, or modify and recalculate cells or return them to their previous state. In addition, you
can use Markdown in documentation cells.

### (Optional) Testing Jupyter notebooks

By default all cells are running Python:

In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

We can also use `!` to run individual lines in the terminal.

In [ ]:
!echo "Hello world"

Here are some useful keyboard shortcuts in notebooks:

* `shift - enter` execute cell and move to the next one
* `alt - enter` execute cell and insert new below
* `esc` exit cursor/edit mode and enter command mode
* `a` add cell above
* `b` add cell below
* `dd` delete cell
* `x` cut selected cells
* `c` copy selected cells
* `v` paste cells below
* `m` change cell to Markdown
* `y` change cell to Code

Try a few below! Markdown cells (such as this one) don't execute code but they **can** contain _nice_ formatting.

They can also include `code` snippets (along with the declaration of the language to get syntax hightlighting which you can see in editing mode):

```python
def hello(name):
    print(f"hello {name}")
```

# And Titles
## Headings
### Subheadings
#### and Sub-subheadings

<div class="alert alert-info">
... and HTML formatting
</div>


... and even LaTex!

$
f(x) = \int_{-\infty}^{\infty} e^{-x^2} dx
$


# Why GRASS GIS in Colab?

Since Colab offers Jupyter notebooks in a Linux environment
**it is really easy to install or even compile GRASS GIS therein**. Also, the integration with Google Drive makes it a great resource to run our workflows in the cloud and export the results or keep our GRASS projects and code there. This facilitates the teaching of workshops as participants do not need to install or download anything on their own computers 🤩

There are a couple of things to consider when working with GRASS GIS within Colab:
- You will need to *install GRASS GIS every time they start a new working session or notebook*.
- The files you download in Colab *are only valid for the current session*.

Alternatively, **users can mount their Google drive**, download data and create their GRASS projects there. Those will be preserved even if the runtime is disconnected or the session closed.

# GRASS GIS basics

**GRASS GIS is a geoprocessing engine with different interfaces.**

The functionality of GRASS can be used from its Graphical User Interface (GUI), from command line (CLI), from a Python IDE or Jupyter Notebook via the [grass.script](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html), [pygrass](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass_index.html) or [grass.jupyter](https://grass.osgeo.org/grass84/manuals/libpython/grass.jupyter.html) packages, from R IDEs like RStudio via the [rgrass](https://github.com/rsbivand/rgrass/) package, from REST APIs like [actinia](https://actinia.mundialis.de/) and also via web processes like WPS. GRASS GIS is a versatile and robust software package.

### Open GRASS for the first time: CLI and GUI

<br>
<a href="img/grass_gui_first_time_and_cli_combined.png">
  <img src="https://github.com/veroandreo/foss4g2024_grass4rs/blob/main/img/grass_gui_first_time_and_cli_combined.png?raw=1"
   alt="First time launching GRASS 8"
   title="First time launching GRASS 8.4"
   width="65%">
</a>

GRASS GUI has a single window layout by default, but it is also possible to minimize and/or dock/undock the panels. On the right, you can find the **data** browser which allows you to navigate through your projects and data, and the **layers** panel showing displayed layers. The panel in the middle is the
**map display**. You can add additional ones if you need using
![](https://github.com/veroandreo/foss4g2024_grass4rs/blob/main/img/monitor-create.png?raw=1).

On the right there are multiple tabs where you can find a searchable **tools' tree** similar to the Processing
toolbox in QGIS, a **console** where you can type GRASS commands, the **history of executed commands** in case you want to re-run a task and a simple **Python console** where you can use the GRASS Python API.

### GRASS projects

GRASS **projects** are simply folders storing your geospatial data with common coordinate reference system (CRS), ensuring
consistency of your data.
At the project level, data is further organized into subprojects called **mapsets**, which you can use to manage different subregions or analyses within a project.
Each project contains a special mapset called *PERMANENT*, which is used to store source datasets for your analysis that can be easily accessed from other mapsets.

By default, GRASS will create a **data folder** named `grassdata` to store projects the first time you open it. However, this is a matter of taste and you can organize your GRASS projects differently.


> More info: <a href="https://grass.osgeo.org/grass-stable/manuals/grass_database.html">https://grass.osgeo.org/grass-stable/manuals/grass_database.html</a>.

### GRASS tools and extensions

GRASS has more than [500 modules](https://grass.osgeo.org/grass-stable/manuals/full_index.html) for the most varied tasks:

| Prefix                                                               | Function class   | Type of command                     | Example
|--------------------------------------------------------------------- |:---------------- |:----------------------------------- |:-------------------------------------------------------------------------------------------------------------------
| [g.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#g)    | general          | general data management             | [g.rename](https://grass.osgeo.org/grass-stable/manuals/g.rename.html): renames map
| [d.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#d)    | display          | graphical output                    | [d.rast](https://grass.osgeo.org/grass-stable/manuals/d.rast.html): display raster map
| [r.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r)    | raster           | raster processing                   | [r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html): map algebra
| [v.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r)    | vector           | vector processing                   | [v.clean](https://grass.osgeo.org/grass-stable/manuals/v.clean.html): topological cleaning
| [i.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#i)    | imagery          | imagery processing                  | [i.pca](https://grass.osgeo.org/grass-stable/manuals/i.pca.html): Principal Components Analysis on imagery group
| [r3.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r3)  | voxel            | 3D raster processing                | [r3.stats](https://grass.osgeo.org/grass-stable/manuals/r3.stats.html): voxel statistics
| [db.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#db)  | database         | database management                 | [db.select](https://grass.osgeo.org/grass-stable/manuals/db.select.html): select value(s) from table
| [ps.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#ps)  | postscript       | PostScript map creation             | [ps.map](https://grass.osgeo.org/grass-stable/manuals/ps.map.html): PostScript map creation
| [t.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#t)    | temporal         | space-time datasets                 | [t.rast.aggregate](https://grass.osgeo.org/grass-stable/manuals/t.rast.aggregate.html): raster time series aggregation

Extensions or **add-ons** can be installed from the
[central add-on GitHub repository](https://grass.osgeo.org/grass-stable/manuals/addons/)
or from *other users' GitHub* (or similar Git repositories) using the command
[g.extension](https://grass.osgeo.org/grass-stable/manuals/g.extension.html). For example:

```bash
 # install an extension from the official GRASS GIS add-on repository
 g.extension extension=r.hants

 # install an extension from another GitHub repository
 g.extension extension=r.change.stats url=https://github.com/mundialis/r.change.stats
```

# GRASS & Python

## Python package `grass.script`

The **grass.script** or GRASS GIS Python Scripting Library provides functions for calling GRASS modules within Python scripts. The most commonly used functions include:

- `run_command`: used when the output of the modules is a raster or vector, no text type output is expected
- `read_command`: used when the output of the modules is of text type
- `parse_command`: used with modules whose output can be converted to `key=value` pairs
- `write_command`: used with modules that expect text input, either in the form of a file or from stdin

It also provides several wrapper functions for frequently used modules, for example:

- To get info from a raster, script.raster.raster_info() is used: `gs.raster_info('dsm')`
- To get info of a vector, script.vector.vector_info() is used: `gs.vector_info('roads')`
- To list the raster in a location, script.core.list_grouped() is used: `gs.list_grouped(type=['raster'])`
- To obtain the computational region, script.core.region() is used: `gs.region()`

> More info: <a href="https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html">https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html</a>

## Python package `grass.jupyter`

The **grass.jupyter** library improves the integration of GRASS and Jupyter, and provides different classes to facilitate GRASS maps visualization:

- `init`: starts a GRASS session and sets up all necessary environment variables
- `Map`: 2D rendering
- `Map3D`: 3D rendering
- `InteractiveMap`: interactive visualization with folium or ipyleaflet
- `SeriesMap` and `TimeSeriesMap`: visualization of spatio-temporal data

> More info: <a href="https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html">https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html</a>

# Install GRASS GIS in Colab

Let's first print system description to know where are we:

In [ ]:
!lsb_release -a

At the time of preparing this workshop, Colab is using Linux
[Ubuntu 22.04.3 LTS](https://medium.com/google-colab/colab-updated-to-ubuntu-22-04-lts-709a91555b3c).
To get a recent GRASS GIS version, we add the [`ubuntugis-unstable`](https://launchpad.net/~ubuntugis/+archive/ubuntu/ubuntugis-unstable) repository, update the package list and install GRASS GIS. It might take a couple of minutes according to the resources available:

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev
print("INSTALLATION COMPLETED")


Check that GRASS GIS is installed by asking which version is now installed (we expect v8.4 or later):

In [ ]:
!grass --config version

### Other dependencies
Some Python dependencies are needed to run the exercises of this notebook. Install as follows:

In [ ]:
!python -m pip install eodag[usgs] folium scikit-learn pandas numpy seaborn matplotlib ipyleaflet

# Set up working directory and download sample data

By default we'll have access to the `/content` folder within Colab, and any data we download will be placed there. In any case, we should bare in mind that whatever data we download within Colab, will disappear if the runtime gets disconected because of inactivity or once we close the Colab session.

Let's define our folder structure and get the [North Carolina sample dataset](https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.zip) and other datasets into Colab.

In [ ]:
import os

# create our data directory
os.makedirs("foss4g_grass4rs", exist_ok=True)
homedir = os.path.join(os.getcwd(), "foss4g_grass4rs")

# define GRASS GIS database variables
grassdata = os.path.join(homedir, "grassdata")
project = "nc_spm_08_grass7"
mapset = "PERMANENT"

Next, we download **North Carolina dataset** and unpack it within the above defined `homedir`.

In [ ]:
# download NC sample data into target directory homedir
!wget -c https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.zip -O $homedir/nc.zip

In [ ]:
# unpack sample dataset into grassdata
!unzip -o -q -d $grassdata $homedir/nc.zip

print("List uploaded file(s) in target directory "+homedir+":")
os.listdir(homedir)

Now, we'll download and unzip the Sentinel-2 scenes we'll use in this exercise:

In [ ]:
# Sentinel-2 related directories
s2_data = os.path.join(homedir, "sentinel")
s2_timestamps = os.path.join(homedir, s2_data, "sentinel-timestamps.txt")

In [ ]:
# download Sentinel-2 data
!wget -c https://data.neteler.org/foss4g2022/sentinel.zip -O $homedir/sentinel.zip

# unpack into target directory
!unzip -o -q -d $homedir $homedir/sentinel.zip

Finally, let's download the landuse map we'll need later on:

In [ ]:
# get NC landuse map 2019 in GRASS GIS format, to be used later as classification training map
!wget -c https://data.neteler.org/foss4g2022/nc_nlcd2019.pack -O $homedir/nc_nlcd2019.pack

and list the content of our working directory:

In [ ]:
print("List uploaded file(s) in target directory "+homedir+":")
os.listdir(homedir)

# Start GRASS GIS

In [ ]:
# import standard Python packages we need
import sys
import subprocess

# ask GRASS GIS where its Python packages are to be able to run it from the notebook
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

In [ ]:
# import the GRASS GIS python packages
import grass.script as gs
import grass.jupyter as gj

# start the GRASS GIS Session
session = gj.init(grassdata, project, mapset)

In [ ]:
# show current GRASS GIS settings
gs.gisenv()

Before we start, we list the elements in the mapset `PERMANENT`. If you only want to see the raster or vector type elements, just change the `type` option in the following command.

In [ ]:
# list vector elements in the PERMANENT mapset
gs.list_grouped(type="vector")

📝 **Your turn!** List all raster maps and then list raster maps matching a name pattern:

## Quick visualizations

GRASS GIS offers static and interactive map visualizations in Jupyter notebooks.

In [ ]:
# 2d map
map = gj.Map(width = 500, use_region=False)
map.d_rast(map="elevation")
map.d_legend(raster="elevation", at="20,80,85,90", flags="b", border_color="none")
map.show()

In [ ]:
# interactive map with folium
mapf = gj.InteractiveMap(width = 500, use_region=False, map_backend="folium")
mapf.add_raster("elevation")
mapf.add_vector("roadsmajor")
mapf.add_layer_control(position = "bottomright")
mapf.show()

In [ ]:
# interactive map with ipyleaflet
mapi = gj.InteractiveMap(width = 500)
mapi.add_raster("elevation")
mapi.add_vector("roadsmajor")
mapi.add_layer_control()
mapi.show()

# Create a new mapset

Next, we create a new mapset to work with this notebook and import Sentinel-2 data.

In [ ]:
# create a new mapset and switch to it
gs.run_command("g.mapset", mapset="sentinel2", flags="c")

In [ ]:
# check current mapset
print(gs.read_command("g.mapset", flags="p"))

In [ ]:
# print accessible mapsets within the project
print(gs.read_command("g.mapsets", flags="p"))

## Define our area of interest

To search for Sentinel 2 images, we need an area of interest. This area can be defined by a vector map or the computational region. Here, will use a map of urban areas that we already have in the `PERMANENT` mapset. Since we are interested in the city of Raleigh, NC, we use the function [v.extract](https://grass.osgeo.org/grass-stable/manuals/v.extract.html) to extract the polygon corresponding to that urban area only.

In [ ]:
import pandas as pd

# check `urbanarea` vector attributes
urban = gs.parse_command("v.db.select", map="urbanarea", format="json")["records"]
df = pd.DataFrame(urban)
df

In [ ]:
# extract Raleigh urban area from `urbanarea` vector map
gs.run_command("v.extract",
               input="urbanarea",
               where="NAME == 'Raleigh'",
               output="urban_area_raleigh")

In [ ]:
# show attributes
gs.vector_db_select('urban_area_raleigh')['values']

We set the computational region to the boundaries of the newly created vector. This will be the bounding box we'll use for the Sentinel scenes search.

In [ ]:
# set the computational region to the extent of Cordoba urban area
region = gs.parse_command("g.region", vector="urban_area_raleigh", flags="g")
region

> ### Computational region
>
> It refers to the boundary configuration of the analysis area and spatial resolution (raster). The **computational region** can be defined and modified with the command [g.region](https://grass.osgeo.org/grass-stable/manuals/g.region.html) to the extent of a vector map, a raster or manually to some area of interest. The *output raster maps will have an extent and spatial resolution equal to the computational region*, while vector maps are always processed at their original extent.
>
> <a href="https://raw.githubusercontent.com/veroandreo/foss4g2022_grass4rs/main/assets/img/region.png">
  <img src="https://raw.githubusercontent.com/veroandreo/foss4g2022_grass4rs/main/assets/img/region.png"
   alt="Computational region"
   title="Computational region"
   width="80%">
</a>
>
> For more details, see the wiki on <a href="https://grasswiki.osgeo.org/wiki/Computational_region">Computational Region</a>

In [ ]:
# display newly created vector
raleigh_map = gj.InteractiveMap(use_region=True, map_backend="folium")
#raleigh_map.add_raster("elev_state_500m")
raleigh_map.add_vector("urban_area_raleigh")
raleigh_map.add_layer_control(position = "bottomright")
raleigh_map.show()

📝 **Your turn!** Check by yourself what happens when we use `use_region=True` vs `use_region=False` in the cell above, with and without a raster map.

# Tools to download and import RS data into GRASS GIS

There are different tools to search, filter, download, import and process remote sensing data in GRASS GIS. Some examples include:
- [i.eodag](https://grass.osgeo.org/grass-stable/manuals/addons/i.eodag.html): Downloads imagery from various providers through the [EODAG](https://eodag.readthedocs.io/en/stable/index.html) API.
- [i.sentinel](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.html): Toolset to download and process Copernicus Sentinel products.
- [i.landsat](https://grass.osgeo.org/grass-stable/manuals/addons/i.landsat.html): Toolset to download and process Landsat TM, ETM and OLI products.
- [i.modis](https://grass.osgeo.org/grass-stable/manuals/addons/i.modis.html): Toolset to download and process MODIS products using pyModis.
- [t.stac](https://grass.osgeo.org/grass-stable/manuals/addons/t.stac.html): Toolset to explore metadata and ingest SpatioTemporal Asset Catalog (STAC) items, collections, and catalogs
- etc.


Let's explore the i.eodag tool, that is also the backend of i.sentinel we'll show later.

In [ ]:
# install i.eodag
gs.run_command("g.extension", extension="i.eodag")

EODAG allows to search and download data from different providers. Let's check them:

In [ ]:
!i.eodag print=providers

Now, let's see which data providers have Sentinel 2 Level 2A data:

In [ ]:
!i.eodag print=providers producttype=S2_MSI_L2A

📝 **Your turn!** Check [i.eodag](https://grass.osgeo.org/grass-stable/manuals/addons/i.eodag.html) manual page and try other search parameters and filters.

Let's do a quick search for Sentinel 2 scenes intersecting our computational region the last 60 days with cloud cover less or equal to 10%. The default provider is *peps*. We can also use *cop_dataspace* to search for S2 data. However, if we then need to download those data, we need to be registered at the [Copernicus Data Space Ecosystem](https://dataspace.copernicus.eu/).

> See [i.eodag](https://grass.osgeo.org/grass-stable/manuals/addons/i.eodag.html) manual page for more details and examples.

In [ ]:
!i.eodag -l producttype=S2_MSI_L2A clouds=10

In [ ]:
!i.eodag -l producttype=S2_MSI_L2A clouds=10 provider=cop_dataspace

📝 **Your turn!** Using the basic example above, try other options. For example, filter by date, change the spatial relationship, etc.

## Sentinel-2 processing overview

The [i.sentinel](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.html) toolbox facilitates searching, filtering, downloading, importing and pre-processing Sentinel data, especially Sentinel 2, from a GRASS GIS session. The toolbox consists of six GRASS addon tools:

* [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html)
* [i.sentinel.import](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.import.html)
* [i.sentinel.preproc](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.preproc.html)
* [i.sentinel.mask](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.mask.html)
* [i.sentinel.coverage](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.coverage.html)
* [i.sentinel.parallel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.parallel.download.html)

Let's install it:

In [ ]:
gs.run_command("g.extension", extension="i.sentinel")

Check if the module is there by running it with optional arguments:

In [ ]:
gs.core.find_program("i.sentinel.download", "--help")

### Sentinel 2 data search and download

We'll use [Sentinel-2 Level 2A (L2A) products](https://sentinels.copernicus.eu/de/web/sentinel/user-guides/sentinel-2-msi/product-types/level-2a) to avoid the need of computing atmospheric corrections.

In order to search and download Sentinel products from the Copernicus Data Space Ecosystem, you need to [register](https://dataspace.copernicus.eu/) first. If you are already registered, create a text file in the data directory (`homedir`) containing two lines: username and password. If you already have this file, upload it in order to have it accessible.

We'll exemplify how to upload the `esa_credentials.txt` file using commands. However, you can also use Colab's interface as below. Choose whatever way feels easier for you.

![](https://github.com/veroandreo/foss4g2024_grass4rs/blob/main/img/upload_or_create_new_file.png?raw=1)

In [ ]:
# upload the file to Colab - code below will generate a "Browse" button for data upload
from google.colab import files
uploaded = files.upload()

!mv "esa_credentials.txt" $homedir
!ls $homedir

Let’s search for the latest available S2 products by means of [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html). Setting the `-l` flag, the result will only
be printed.

In [ ]:
# list available Sentinel-2 L2A scenes for AOI
# note that we use parse_command() in order to intercept the output
gs.parse_command("i.sentinel.download",
                 flags="l",
                 producttype="S2MSI2A",
                 map="urban_area_raleigh",
                 settings=os.path.join(homedir, "esa_credentials.txt"))

By default, the tool returns all the products meeting the defined criteria for the last 60 days. Let’s change
the search period setting `start` and `end` options. We will also limit products by `clouds` coverage percentage
threshold and `sort` them by ingestion date.

In [ ]:
gs.parse_command("i.sentinel.download",
                 flags="l",
                 producttype="S2MSI2A",
                 map="urban_area_raleigh",
                 settings=os.path.join(homedir, "esa_credentials.txt"),
                 start="2024-08-31",
                 end="2024-10-31",
                 clouds="5",
                 sort="ingestiondate",
                 limit=10)

> If a long list of products have been found, you can limit the amount with the <code>limit</code> option as we did above.

Let's save the output of the search into a list and then beautify the display by creating a pandas table.

In [ ]:
list_prod = gs.read_command("i.sentinel.download",
                            flags="l",
                            producttype="S2MSI2A",
                            map="urban_area_raleigh",
                            settings=os.path.join(homedir, "esa_credentials.txt"),
                            footprints="s2_footprints", # we save the footprints in a vector file
                            start="2024-08-31",
                            end="2024-10-31",
                            clouds="5",
                            sort="ingestiondate",
                            limit=10)

In [ ]:
# print plain list
list_prod

In [ ]:
from io import StringIO

pd.read_csv(StringIO(list_prod), delimiter=" ", usecols=[0, 1, 4, 5],
            names=['scene', 'date', 'cloud', 'product'])

If we want some extra info about each scene before deciding which one to download, we can hava a look at the attribute table of the footprint vector we saved.

In [ ]:
# check `footprints` vector attributes
footprints = gs.parse_command("v.db.select", map="s2_footprints", format="json")["records"]
df = pd.DataFrame(footprints)
df

Let's also visualize the footprints map.

> In the upcoming GRASS 8.5 the interactive map class comes with several improvements, that will for example, allow us to query the attributes of each footprint.

In [ ]:
# diplay footprints (you may want to zoom out a bit)
fp_map = gj.InteractiveMap(width = 400, use_region=True)
fp_map.add_vector("s2_footprints")
fp_map.add_vector("urban_area_raleigh")
fp_map.add_layer_control(position = "bottomright")
fp_map.show()

The next step is to download the scene or scenes of interest. Just remove the `-l` flag and add the `output` option in order to define the path to the output directory where data should be saved.

As download might take quite some time, we'll **skip this part** and directly use an already prepared set of smaller, ready to import scenes which we downloaded above. Still, we leave an example below for future reference :)

Go to section **"Importing Sentinel 2 data"**

In [ ]:
# download example:
# gs.run_command("i.sentinel.download",
#               settings=s2_credentials,
#               id="S2B_MSIL2A_20240921T154849_N0511_R054_T17SQA_20240921T212218",
#               output=s2_data)

### Importing Sentinel-2 data

Before importing or linking Sentinel-2 data we print a list of filtered raster files including projection match (1 for match, otherwise 0). If the CRS of the input data differs from that of the current project, you should consider reprojection (`-r` flag) or creating a new project for import.

We'll limit the S2 data import to the RGB and NIR bands (2, 3, 4, 8A) in 10 m spatial resolution using the `pattern` option. Let's check CRS information of the selected bands only.

In [ ]:
# print only to test band selection
gs.parse_command("i.sentinel.import",
                 flags="p",
                 input=s2_data,
                 pattern="B(02|03|04|08)_10m")

> We see that CRS does not match. But how do we know which is the CRS of our current project?
>
> ```
> g.proj -p
> ```

By default, input data are imported into GRASS and converted into GRASS native data format.
Alternatively, data can be linked if the `-l` flag is provided. It is also
useful to import cloud mask vector features (`-c` flag). In addition, we'll use the
`register_output` option to produce a timestamp plain text file
which will be used later on to create a time series.

In [ ]:
# for S2 import, allow for using 2GB of RAM for faster operations.
# (s2_data and s2_timestamps are defined above)
# this may take a few minutes...
gs.parse_command("i.sentinel.import",
                 flags="rcsj",
                 input=s2_data,
                 pattern="B(02|03|04|08)_10m",
                 memory=2000,
                 extent="input",
                 register_output=s2_timestamps)

In [ ]:
# list imported raster maps
gs.list_grouped(type="raster")['sentinel2']

In [ ]:
# check metadata of one of the imported bands
gs.raster_info(map="T17SQV_20220617T155829_B03_10m")["comments"]

In [ ]:
# print timestamp file for inspection
with open(s2_timestamps, 'r') as f:
    content = f.read()
    print(content)
    f.close()

> **Semantic labels**
>
> A fairly new concept within GRASS GIS is semantic labels. Such labels are useful for satellite imagery as they allow us to identify to which sensor and band a given raster corresponds to. They are particularly relevant when working with image collections and also when classifying different scenes. A spectral signature generated for a given set of bands, can then be re-used to classify another scene as long as the semantic labels are the same.
>
> Be ware – although it is possible to re-use spectral signatures to any scene with the same bands, temporal changes (seasons, weather impact) might limit their applicability only to scenes obtained more or less at the same time.

### Visualization

In [ ]:
# create Map instance
b3_map = gj.Map(width=400)
# add a raster, vector and legend to the map
b3_map.d_rast(map="T17SQV_20220617T155829_B03_10m")
b3_map.d_legend(raster="T17SQV_20220617T155829_B03_10m",
                title="Reflectance",
                fontsize=10, at=(70, 93, 80, 90), flags="b")
b3_map.d_barscale()
# display map
b3_map.show()

In [ ]:
# set color table of bands 4, 3 and 2 to grey
gs.run_command("r.colors",
               map="T17SQV_20220617T155829_B04_10m,T17SQV_20220617T155829_B03_10m,T17SQV_20220617T155829_B02_10m",
               color="grey")

In [ ]:
# color enhancing for RGB composition
gs.run_command("i.colors.enhance",
               red="T17SQV_20220617T155829_B04_10m",
               green="T17SQV_20220617T155829_B03_10m",
               blue="T17SQV_20220617T155829_B02_10m",
               strength=92)

In [ ]:
# set region to "elevation" map and align to the S2 data
gs.run_command("g.region",
               raster="elevation",
               align="T17SQV_20220617T155829_B04_10m",
               flags="p")

In [ ]:
# display the enhanced RGB combination
rgb = gj.Map(width=400, use_region=True)
rgb.d_rgb(red="T17SQV_20220617T155829_B04_10m",
          green="T17SQV_20220617T155829_B03_10m",
          blue="T17SQV_20220617T155829_B02_10m")
rgb.show()

## Spectral indices of vegetation and water

We will use *i.vi* to estimate NDVI and NDWI vegetation and water indices. See [i.vi](https://grass.osgeo.org/grass-stable/manuals/i.vi.html) for other available indices.

In [ ]:
# estimate vegetation indices
gs.run_command("i.vi",
               red="T17SQV_20220528T155819_B04_10m",
               nir="T17SQV_20220528T155819_B08_10m",
               output="T17SQV_20220528T155819_NDVI_10m",
               viname="ndvi")

# add semantic label
gs.run_command("r.support",
               map="T17SQV_20220528T155819_NDVI_10m",
               semantic_label="S2_NDVI")

In [ ]:
# estimate water indices
gs.run_command("i.vi",
               green="T17SQV_20220528T155819_B03_10m",
               nir="T17SQV_20220528T155819_B08_10m",
               output="T17SQV_20220528T155819_NDWI_10m",
               viname="ndwi")

# set ndwi color palette
gs.run_command("r.colors", map="T17SQV_20220528T155819_NDWI_10m", color="ndwi")

# add semantic label
gs.run_command("r.support",
               map="T17SQV_20220528T155819_NDWI_10m",
               semantic_label="S2_NDWI")

In [ ]:
# check metadata of NDVI
gs.raster_info(map="T17SQV_20220528T155819_NDVI_10m")["semantic_label"]

In [ ]:
# interactive maps
idx_map = gj.InteractiveMap(width = 400, use_region=True)
idx_map.add_raster("T17SQV_20220528T155819_NDVI_10m", opacity=0.7)
idx_map.add_raster("T17SQV_20220528T155819_NDWI_10m", opacity=0.7)
idx_map.add_layer_control(position = "bottomright")
idx_map.show()
# ... use the layer selector in the corner to enable/disable the NDVI/NDWI layers

#### GRASS GIS maps as numpy arrays

GRASS maps can be read as numpy arrays thanks to the array function of the grass.script library. This facilitates many operations with python libraries that require an array as input. In this case, we demonstrate its use plotting an histogram.

In [ ]:
# import required libraries
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from grass.script import array as garray

# read NDVI and NDWI as numpy arrays
ndvi = garray.array(mapname="T17SQV_20220528T155819_NDVI_10m", null="nan")
ndwi = garray.array(mapname="T17SQV_20220528T155819_NDWI_10m", null="nan")
print(ndvi.shape,ndwi.shape)

In [ ]:
# Plot NDVI and NDWI
sns.set_style('darkgrid')
fig, axs = plt.subplots(1, 2, figsize=(7, 7))
sns.histplot(ax=axs[0], data=ndvi.ravel(), kde=True, color="olive")
sns.histplot(ax=axs[1], data=ndwi.ravel(), kde=True, color="skyblue")
plt.show()

## NDVI time series data processing

### A few concepts of time series data processing in GRASS GIS

GRASS GIS offers specialized tools for spatio-temporal data
processing, see GRASS documentation [temporalintro](https://grass.osgeo.org/grass-stable/manuals/temporalintro.html) for details and the [temporal data processing](https://grasswiki.osgeo.org/wiki/Temporal_data_processing) wiki for examples and a workflow tutorial.

GRASS introduces three special data types that are designed to handle time-series:

* *Space-time raster datasets* (`strds`) for managing raster map time series.

* *Space-time 3D raster datasets* (`str3ds`) for managing 3D raster map time series.

* *Space-time vector datasets* (`stvds`) for managing vector map time series.
  
<a href="https://raw.githubusercontent.com/veroandreo/foss4g2022_grass4rs/main/assets/img/tgrass_flowchart.png">
  <img src="https://raw.githubusercontent.com/veroandreo/foss4g2022_grass4rs/main/assets/img/tgrass_flowchart.png"
   alt="TGRASS flowchart"
   title="GRASS flowchart"
   width="65%">
</a>


### Create space-time dataset

At this moment a new space-time dataset can be created by means of [t.create](https://grass.osgeo.org/grass-stable/manuals/t.create.html) and all imported Sentinel bands registered with [t.register](https://grass.osgeo.org/grass-stable/manuals/t.register.html) and the timestamps file we created when we imported S2 bands.

In [ ]:
gs.run_command("t.create",
               output="s2_nc",
               title="Sentinel L2A - North Carolina",
               desc="Tile T17SQV - 2022")

gs.run_command("t.register",
               input="s2_nc",
               file=s2_timestamps)

Let’s check basic metadata with [t.info](https://grass.osgeo.org/grass-stable/manuals/t.info.html) and list the registered maps with [t.rast.list](https://grass.osgeo.org/grass-stable/manuals/t.rast.list.html).

In [ ]:
# print time series info
print(gs.read_command("t.info", input="s2_nc"))

In [ ]:
# list registered bands in the space-time cube
print(gs.read_command("t.rast.list",
                      input="s2_nc",
                      columns="name,start_time,semantic_label"))

We'll now use a special syntaxis to list only band 4 raster maps withing the time series:

In [ ]:
# list only band 4 maps
print(gs.read_command("t.rast.list",
                      input="s2_nc.S2_4",
                      columns="name,start_time,semantic_label"))

### NDVI Space-Time computation

For NDVI computation the 4th and 8th bands are required, as we saw above for a single map.
Now, we will create a time series of NDVI maps. We will take advantage of the semantic labels syntax and use
[t.rast.mapcalc](https://grass.osgeo.org/grass-stable/manuals/t.rast.mapcalc.html) to estimate NDVI for all the timestamps in the time series, using band 4 and 8 subsets.

In [ ]:
gs.run_command("t.rast.mapcalc",
               inputs="s2_nc.S2_8,s2_nc.S2_4",
               output="s2_ndvi",
               basename="s2_ndvi",
               expression="float(s2_nc.S2_8 - s2_nc.S2_4) / (s2_nc.S2_8 + s2_nc.S2_4)")

When computation is finished, the *ndvi* color table can be set with [t.rast.colors](https://grass.osgeo.org/grass-stable/manuals/t.rast.colors.html):

In [ ]:
gs.run_command("t.rast.colors", input="s2_ndvi", color="ndvi")

In [ ]:
print(gs.read_command("t.info", input="s2_ndvi"))

### Time series plots

Let’s check content of the new dataset by means of [t.rast.list](https://grass.osgeo.org/grass-stable/manuals/t.rast.list.html):

In [ ]:
print(gs.read_command("t.rast.list",
                      input="s2_ndvi",
                      columns="name,start_time,min,max"))

If we save the previous output to a file, we can then plot the min and max time series:

In [ ]:
gs.run_command("t.rast.list",
                input="s2_ndvi",
                columns="name,start_time,min,max",
                format="csv",
                separator="comma",
                output=os.path.join(homedir,"ndvi.csv"))

In [ ]:
# read the csv and plot
ndvi = pd.read_csv(os.path.join(homedir,"ndvi.csv"))
ndvi.plot(0, [2,3], subplots=False)

We could also use [t.rast.univar](https://grass.osgeo.org/grass-stable/manuals/t.rast.univar.html) to obtain extended statistics:

In [ ]:
# get extended univar stats and save them as a csv file
gs.run_command("t.rast.univar",
                flags="e",
                input="s2_ndvi",
                output=os.path.join(homedir,"ndvi_ext_stats.csv"),
                separator="comma")

In [ ]:
# read the csv and plot
ndvi = pd.read_csv(os.path.join(homedir,"ndvi_ext_stats.csv"))
ndvi['start'] = pd.to_datetime(ndvi.start, format="%Y-%m-%d", exact=False)
ndvi.plot.line(1, [3,4,5], subplots=False)

### Query time series in a single point

`g.region` command allows us to get the coordinates of the center of the computational region, we'll use those to query the NDVI time series.

In [ ]:
# get region center coordinates for query (center_easting, center_northing values)
gs.region(complete=True)

In [ ]:
# query map at center coordinates
print(gs.read_command("t.rast.what",
                      strds="s2_ndvi",
                      coordinates="637500,221750",
                      layout="col",
                      flags="n"))

### Time series animation

In [ ]:
# get the list of map names within the strds
import json
result = json.loads(
    gs.read_command(
        "t.rast.list", input="s2_ndvi", format="json"
    )
)

names = [item['name'] for item in result['data']]
names

In [ ]:
# display newly created NDVI time series map
ndviseries = gj.SeriesMap(use_region=True)
ndviseries.add_rasters(names)
ndviseries.d_barscale()
ndviseries.show()

## Create an image stack (imagery group)

**Stack of maps = imagery group**

We use [i.group](https://grass.osgeo.org/grass-stable/manuals/i.group.html)  to stack raster maps (e.g., R-G-B channels or more) in GRASS GIS. These groups/stacks are just based on metadata, so they do not take up more disk space.

In [ ]:
# list of selected S2 maps
s2_maps = gs.list_grouped(type="raster", pattern="*20220528T155819*")['sentinel2']
print(s2_maps)

In [ ]:
# create group and subgroup with S2 bands
gs.run_command("i.group", group="s2", subgroup="s2", input=s2_maps)
print(gs.read_command("i.group", group="s2", flags="l"))

## Object recognition with image segmentation

We'll use [i.segment](https://grass.osgeo.org/grass-stable/manuals/i.segment.html) to perform image segmentation. The resulting map will be used together with S2 bands, NDVI and NDWI to perform supervised classification.

In [ ]:
# threshold = 0 merges only identical segments; threshold = 1 merges all
gs.run_command("i.segment",
               group="s2",
               threshold="0.05",
               minsize="100",
               output="sentinel_segments_min100",
               goodness="sentinel_segments_goodness_min100",
               memory=2000)

In [ ]:
# display newly created segments raster map
segments = gj.InteractiveMap(width = 400, use_region=True)
segments.add_raster("sentinel_segments_min100", opacity=0.8)
segments.add_raster("s2_ndvi_4", opacity=0.3)
segments.add_layer_control(position = "bottomright")
segments.show()

In [ ]:
# show univariate statistics of goodness-of-fit raster map, with extended statistics (quartiles)
print(gs.read_command("r.univar",
                      map="sentinel_segments_goodness_min100",
                      flags="ge"))

In [ ]:
# assign color table (low fit values: blue; high fit values: green)
gs.run_command("r.colors",
               map="sentinel_segments_goodness_min100",
               color="byg",
               flags="e")

In [ ]:
# display newly created goodness-of-fit raster map
segments = gj.InteractiveMap(width = 400, use_region=True)
segments.add_raster("sentinel_segments_goodness_min100", opacity=0.8)
segments.add_layer_control(position = "bottomright")
segments.show()

> If you are interested in Object Based Image Analysis (OBIA), check these  addon tools and the references therein:
> - [i.segment.uspo](https://grass.osgeo.org/grass-stable/manuals/addons/i.segment.uspo.html) for unsupervised parameter optimization
> - [i.segment.stats](https://grass.osgeo.org/grass-stable/manuals/addons/i.segment.stats.html) to extract stats from segments

## Supervised Classification: RandomForest

We will now demonstrate a very much simplified workflow to perform a supervised [Random Forest classification](https://en.wikipedia.org/wiki/Random_forest).

We will feed the following data into the model:

- NDVI and NDWI maps (created above)
- image segmentation (created above)
- random training points extracted from landuse map

First we inspect the raster maps available in the current mapset (i.e., `sentinel2`), just to recall their names.

In [ ]:
gs.list_grouped(type="raster")["sentinel2"]

### Creation of a classification training map by sampling from existing data

In order to generate training data for the Sentinel-2 image classification, we will use the [National Land Cover Database (NLCD) 2019](https://www.lib.ncsu.edu/gis/lulc). It is available for download (30m raster map) from [here](https://drive.google.com/open?id=18D99kuotQp_BkxBnkn8OS3qgCeLVwovb&authuser=0). However, we have already prepared the dataset (`nc_nlcd2019.pack` landuse map). We will use it to perform stratified sampling to retrieve training and test data.

In [ ]:
# import the North Carolina NLCD2019 raster map (subset; resampled to 10m)
gs.run_command("r.unpack",
               input=os.path.join(homedir, "nc_nlcd2019.pack"))

In [ ]:
# check raster categories of landuse map
print(gs.read_command("r.category",
                      map="nc_nlcd2019",
                      separator="comma"))

In [ ]:
# display nc_nlcd2019 landuse raster map
lulc = gj.InteractiveMap(width = 400, use_region=True)
lulc.add_raster("nc_nlcd2019", opacity=0.6)
lulc.add_layer_control(position = "bottomright")
lulc.show()

We already note differences between the underlying OpenStreetMap data and the 30m NLCD map, but well...

In [ ]:
# show simple legend
legend = gj.Map(width=400, use_region=True)
# at=bottom,top,left,right, percentage of screen coordinates (0,0 is lower left)
legend.d_legend(raster="nc_nlcd2019",
                title="Classes",
                fontsize=12,
                at=(10, 90, 20, 90),
                flags="n")
legend.show()

### Random sampling from rasterized simplified landuse map

We now perform stratified sampling, i.e. we extract for each land use class `n` sampling points, using the GRASS GIS addon [r.sample.category](https://grass.osgeo.org/grass-stable/manuals/addons/r.sample.category.html).

First, we install this addon.

In [ ]:
# install r.sample.category
gs.run_command("g.extension", extension="r.sample.category")

In [ ]:
# stratified random sampling to generate points
gs.run_command("r.sample.category",
               input="nc_nlcd2019",
               output="landuse_random_points",
               n="150",
               random_seed="43")

In [ ]:
# visualize newly created vector points map
points = gj.InteractiveMap(width = 400, use_region=True)
points.add_raster("nc_nlcd2019", opacity=0.7)
points.add_vector("landuse_random_points")
points.add_layer_control(position = "bottomright")
points.show()

In [ ]:
# check `random_points` vector attributes
rp = gs.parse_command("v.db.select", map="landuse_random_points", format="json")["records"]
df = pd.DataFrame(rp)
df

Before proceeding with the training of a RF model, we'll split our set of random points into train and test sets. For that, we'll use the [v.divide.training_validation](https://mundialis.github.io/v.divide.training_validation/) tool. Note that the addon is not in the oficial GRASS addons repo, but at [mundialis](https://github.com/mundialis/)'s.

In [ ]:
# install the extension
gs.run_command("g.extension",
               extension="v.divide.training_validation",
               url="https://github.com/mundialis/v.divide.training_validation")

In [ ]:
# split random points into train and test
gs.run_command("v.divide.training_validation",
               input="landuse_random_points",
               column="label",
               training="train",
               validation="test",
               training_percent=70)

📝 **Your turn!** Check the attribute table of *train* and *test* vector maps.

Since RF classifier expects raster points as input, we convert the *train* vector map using [v.to.rast](https://grass.osgeo.org/grass-stable/manuals/v.to.rast.html).

In [ ]:
# convert points from vector to raster model
gs.run_command("v.to.rast",
               input="train",
               output="train",
               use="attr",
               attribute_column="nc_nlcd2019",
               label_column="label")

In [ ]:
# check raster categories of new raster training map
# skip reporting on empty cells
print(gs.read_command("r.report",
                      map="train",
                      flags="n"))

### Train RF model

First we install the [r.learn.ml2](https://grass.osgeo.org/grass-stable/manuals/addons/r.learn.ml2.html) extention. It consists of two modules: `r.learn.train` and `r.learn.predict`.

In [ ]:
# install ML extension
gs.run_command("g.extension", extension="r.learn.ml2")

In [ ]:
# add segmentation map to group and subgroup already populated with S2 bands, NDWI and NDVI
gs.run_command("i.group",
               group="s2",
               subgroup="s2",
               input="sentinel_segments_min100")

# list group content
print(gs.read_command("i.group", group="s2", flags="l"))

We now train the ML model using [r.learn.train](https://grass.osgeo.org/grass-stable/manuals/addons/r.learn.train.html), with model "RandomForestClassifier".

In [ ]:
# train a random forest classification model
gs.run_command("r.learn.train",
               group="s2",
               training_map="train",
               model_name="RandomForestClassifier",
               save_model=os.path.join(homedir, "rf_model.gz"))

 The model has been stored in the file `rf_model.gz` for use in the prediction step of the supervised classification.

In [ ]:
os.listdir(homedir)

### Predict RF model trained

The trained model will now be applied to the entire dataset.

In [ ]:
# predict using r.learn.predict
gs.run_command("r.learn.predict",
               group="s2",
               load_model=os.path.join(homedir, "rf_model.gz"),
               output="sentinel_rf")

In [ ]:
# set color table, we transfer the colors from the original landuse map
gs.run_command("r.colors",
               map="sentinel_rf",
               raster="nc_nlcd2019")

### Visualize and report

In [ ]:
# visualize classified `sentinel_rf` map
rfmap = gj.InteractiveMap(width = 600)
rfmap.add_raster("sentinel_rf", opacity=0.7)
# rfmap.add_raster("nc_nlcd2019", opacity=0.7)
rfmap.add_layer_control(position = "bottomright")
rfmap.show()

In [ ]:
# show legend
legend = gj.Map(width=400, use_region=True)
legend.d_legend(raster="sentinel_rf",
                title="Classes",
                font="sans",
                fontsize=14,
                at=(10, 80, 10, 40),
                flags="n")
legend.show()

In [ ]:
# show class distribution in percent
print(gs.read_command("r.report",
                      map="sentinel_rf",
                      units="p",
                      flags="h"))

In [ ]:
# export map to GTiff (COG requires GDAL 3.1)
gs.run_command("r.out.gdal",
               flags="fmt", #
               input="sentinel_rf",
               output=os.path.join(homedir, "nc_sentinel2_RF.tif"),
               format="GTiff",
               overviews="4")

### Evaluate results

We will now do an evaluation of our results, using the [r.kappa](https://grass.osgeo.org/grass-stable/manuals/r.kappa.html) tool to calculate different accuracy measures. For this, we'll use the *test* dataset that we separated earlier.

In [ ]:
# convert labeled test segments to raster
gs.run_command("v.to.rast",
               input="test",
               output="test",
               use="attr",
               attribute_column="nc_nlcd2019",
               label_column="label")

In [ ]:
# create confusion matrix and estimate precision measures
print(gs.read_command("r.kappa",
                      classification="sentinel_rf",
                      reference="test",
                      flags="w"))

## Bonus track: Supervised Classification with Maximum Likelihood and use of semantic labels

We will now demonstrate the workflow to perform a supervised maximum likelihood classification which is integrated with the semantic labels metadata class, and hence allow us to use the same spectral signature to classify multiple scenes as long as the raster map order in the group is the same.

Let's first check the semantic labels of the bands in our `s2` group:

In [ ]:
band_list = gs.read_command("i.group", group="s2", flags="lg")

In [ ]:
# add semantic label to the segmentation
gs.run_command("r.support",
               map="sentinel_segments_min100",
               semantic_label="S2_seg")

In [ ]:
for m in band_list.split():
    sl = gs.raster_info(m)['semantic_label']
    print(m,sl)

Now, we generate the signature file based on the training sample that we obtained earlier, this will then be the input for the maximum likelihood classification

In [ ]:
# obtain signature files
gs.run_command("i.gensig",
               trainingmap="train",
               group="s2",
               subgroup="s2",
               signaturefile="sig_sentinel")

In [ ]:
# perform ML supervised classification
gs.run_command("i.maxlik",
               group="s2",
               subgroup="s2",
               signaturefile="sig_sentinel",
               output="sentinel_maxlik")

In [ ]:
# check classes
print(gs.read_command("r.category",
                      map="sentinel_maxlik",
                      separator="comma"))

In [ ]:
# set color table, we transfer the colors from the original landuse map
gs.run_command("r.colors",
               map="sentinel_maxlik",
               raster="nc_nlcd2019")

In [ ]:
# display results
maxlik_sup_class = gj.Map(width=500, use_region=True)
maxlik_sup_class.d_rast(map="sentinel_maxlik")
maxlik_sup_class.d_legend(raster="sentinel_maxlik",
                          title="Class",
                          fontsize=12,
                          at=(50, 95, 65, 90),
                          flags="bn")
maxlik_sup_class.d_barscale()
maxlik_sup_class.show()

In [ ]:
# percentage of each class
print(gs.read_command("r.report",
                      map="sentinel_maxlik",
                      units="p",
                      flags="h"))

In [ ]:
# class statistics: NDVI
class_stats = gs.read_command("r.univar",
                              map="T17SQV_20220528T155819_NDVI_10m",
                              zones="sentinel_maxlik",
                              flags="t")

In [ ]:
pd.read_csv(StringIO(class_stats),
            delimiter="|",
            usecols=[1, 4, 5, 7])

Next, and to demonstrate the use of semantic labels, we will classify another sentinel scene with the same signature obtained earlier. To this aim, we need to:
1. create a new imagery group for a different scene with the exact same band order
1. estimate NDVI and NDWI and assign semantic labels
1. run a segmentation and assign semantic labels
1. check group and semantic labels
1. run `i.maxlik`

<div class="alert alert-warning">Be ware – changes over time (phenology, weather) will make spectral signatures to not fit well or at all. Do not use same signatures for a different season!

In [ ]:
s2_maps = gs.list_grouped(type="raster", pattern="*20220617*")['sentinel2']
s2_maps

In [ ]:
# since imagery groups can not be overwritten,
# we delete any leftover "s2_new" group from previous runs
gs.run_command("g.remove",
               type="group",
               name="s2_new",
               flags="f")

In [ ]:
gs.run_command("i.group", group="s2_new", subgroup="s2_new", input=s2_maps)
print(gs.read_command("i.group", group="s2_new", flags="l"))

In [ ]:
# estimate NDVI
gs.run_command("i.vi",
               red="T17SQV_20220617T155829_B04_10m",
               nir="T17SQV_20220617T155829_B08_10m",
               output="T17SQV_20220617T155829_NDVI_10m",
               viname="ndvi")

# add semantic label
gs.run_command("r.support",
               map="T17SQV_20220617T155829_NDVI_10m",
               semantic_label="S2_NDVI")

In [ ]:
# estimate NDWI
gs.run_command("i.vi",
               green="T17SQV_20220617T155829_B03_10m",
               nir="T17SQV_20220617T155829_B08_10m",
               output="T17SQV_20220617T155829_NDWI_10m",
               viname="ndwi")

# add semantic label
gs.run_command("r.support",
               map="T17SQV_20220617T155829_NDWI_10m",
               semantic_label="S2_NDWI")

In [ ]:
# add NDVI and NDWI to s2_mew group
gs.run_command("i.group",
               group="s2_new",
               subgroup="s2_new",
               input="T17SQV_20220617T155829_NDVI_10m,T17SQV_20220617T155829_NDWI_10m")

# print maps in the group
print(gs.read_command("i.group", group="s2_new", flags="l"))

In [ ]:
# run segmentation
gs.run_command("i.segment",
               group="s2_new",
               threshold="0.05",
               minsize="100",
               output="sentinel_new_segments_min100",
               goodness="sentinel_new_segments_goodness_min100")

In [ ]:
# add semantic label to the segmentation
gs.run_command("r.support",
               map="sentinel_new_segments_min100",
               semantic_label="S2_seg")

In [ ]:
# add segmentation to the s2_new group
gs.run_command("i.group", group="s2_new", subgroup="s2_new", input="sentinel_new_segments_min100")

In [ ]:
# check
print(gs.read_command("i.group", group="s2_new", flags="l"))

In [ ]:
# run the classification
gs.run_command("i.maxlik",
               group="s2_new",
               subgroup="s2_new",
               signaturefile="sig_sentinel",
               output="sentinel_maxlik_new")

In [ ]:
# set color table, we transfer the colors from the original landuse map
gs.run_command("r.colors",
               map="sentinel_maxlik_new",
               raster="nc_nlcd2019")

In [ ]:
# display results
maxlik_sup_class = gj.Map(width=500, use_region=True)
maxlik_sup_class.d_rast(map="sentinel_maxlik_new")
maxlik_sup_class.d_legend(raster="sentinel_maxlik_new",
                          title="Class",
                          fontsize=12,
                          at=(60, 95, 70, 90),
                          flags="bn")
maxlik_sup_class.d_barscale()
maxlik_sup_class.show()

# What's next?

You may enjoy more Jupyter notebooks at: https://github.com/OSGeo/grass/tree/main/doc/notebooks

# References

- [GRASS GIS Reference Manual](https://grass.osgeo.org/grass-stable/manuals/)
- [GRASS GIS Addons Reference Manuals](https://grass.osgeo.org/grass-stable/manuals/addons/)
- [GRASS GIS Python library documentation](https://grass.osgeo.org/grass-stable/manuals/libpython/)
- [Temporal data processing](https://grasswiki.osgeo.org/wiki/Temporal_data_processing)